In [1]:
import geopandas as gpd
import pandas as pd
import fiona
import multiprocess as mp
import tqdm
import shapely
shapely.speedups.enable()

In [2]:
# read joined up workplace zone
wkz=gpd.read_file('workplacezoneUK.geojson')

In [29]:
# load up EW workplace zone, drop and rename columns, change CRS
wkzEW = gpd.read_file('/Users/Lauh/Documents/GitHub/postcode-to-workplace-zone/Workplace_Zones_(December_2011)_Boundaries_EW.geojson')
wkzEW=wkzEW.drop(columns=['objectid','lad11nm','lad11nmw','st_areashape','st_lengthshape'])
wkzEW.rename(columns={'wz11cd':'WZCD','lad11cd':'LADCD'},inplace=True)
wkzEW=wkzEW.to_crs('WGS84')

# load up SC workplace zone and change CRS
wkzSC=gpd.read_file('/Users/Lauh/Documents/GitHub/postcode-to-workplace-zone/WorkplaceZones2011Scotland/WorkplaceZones2011Scotland.shp')
wkzSC=wkzSC.to_crs('WGS84')

# load up NI workplace zone and change CRS
wkzNI=gpd.read_file('zip://WZ2011_NI_ESRI_0.zip')
wkzNI.rename(columns={'CD':'WZCD'},inplace=True)
wkzNI=wkzNI.to_crs('WGS84')

# try and join them all together, save as big ass file
wkz=gpd.GeoDataFrame(pd.concat([wkzEW,wkzNI,wkzSC]))
# wkz.to_file("workplacezoneUK.geojson", driver='GeoJSON')

In [56]:
# read in classifications and join info onto geojson
wkzclassification=pd.read_csv('classifications.csv')

In [41]:
# read in postcodes, make it geodataframe, then set the CRS
pc = pd.read_csv('/Users/Lauh/Documents/GitHub/postcode-to-workplace-zone/ONSPD_AUG_2021_UK.csv')
pc = gpd.GeoDataFrame(pc,geometry=gpd.points_from_xy(pc['long'],pc['lat']))
pc = pc.set_crs('WGS84')

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (18,31,40,45) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [45]:
# sjoin postcodes and workplace zone
pcwkz=gpd.sjoin(pc, wkz,op='within')

In [58]:
# left merge the workplace zone classification
pcwkzclass=pcwkz.merge(wkzclassification,how='left',left_on='WZCD',right_on='Workplace Zone Code')

In [62]:
pcwkzclass.columns

Index(['pcd', 'pcd2', 'pcds', 'dointr', 'doterm', 'oscty', 'ced', 'oslaua',
       'osward', 'parish', 'usertype', 'oseast1m', 'osnrth1m', 'osgrdind',
       'oshlthau', 'nhser', 'ctry', 'rgn', 'streg', 'pcon', 'eer', 'teclec',
       'ttwa', 'pct', 'itl', 'statsward', 'oa01', 'casward', 'park', 'lsoa01',
       'msoa01', 'ur01ind', 'oac01', 'oa11', 'lsoa11', 'msoa11', 'wz11', 'ccg',
       'bua11', 'buasd11', 'ru11ind', 'oac11', 'lat', 'long', 'lep1', 'lep2',
       'pfa', 'imd', 'calncv', 'stp', 'geometry', 'index_right', 'WZCD',
       'LADCD', 'Workplace Zone Code', 'Supergroup Code', 'Supergroup Name',
       'Group Code', 'Group Name'],
      dtype='object')

In [68]:
pcclean=pcwkzclass.drop(columns=['dointr', 'doterm', 'oscty', 'ced', 'oslaua',
       'osward', 'parish', 'usertype', 'oseast1m', 'osnrth1m', 'osgrdind',
       'oshlthau', 'nhser', 'ctry', 'rgn', 'streg', 'pcon', 'eer', 'teclec',
       'ttwa', 'pct', 'itl', 'statsward', 'oa01', 'casward', 'park', 'lsoa01',
       'msoa01', 'ur01ind', 'oac01', 'oa11', 'lsoa11', 'msoa11', 'wz11', 'ccg',
       'bua11', 'buasd11', 'ru11ind', 'oac11', 'lat', 'long', 'lep1', 'lep2',
       'pfa', 'imd', 'calncv', 'stp', 'geometry', 'index_right','LADCD','WZCD'])

In [72]:
pcclean = pd.DataFrame(pcclean)
pcclean.to_csv('postcode-workplacezone-lookup.csv',index=False)